In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, TimeDistributed, GlobalAveragePooling1D, Add, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics
import os
import random
import tensorflow as tf

In [2]:
# reset the seeds for reproducibility
def reset_random_seeds():
    n = 1
    os.environ['PYTHONHASHSEED'] = str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    random.seed(n)

reset_random_seeds()

# import data
df = pd.read_csv('Boruta_TA_data.csv')
df1 = pd.read_csv('all_data.csv')

#df = df[df['timestamp'] >= '2013-03-11'].reset_index(drop=True)

#onchain_data_new
#TA_data
#all_data
#Boruta_data
#Boruta_onchain_data
#Boruta_TA_data

In [3]:
# separate the inputs and target
#X = df.drop('timestamp', axis=1)
X = df1['price-ohlc-usd-c']

# create binary classification for price movement
price = pd.DataFrame()
price['today'] = df1['price-ohlc-usd-c']
price['next day'] = price['today'].shift(-1)
y = (price['next day'] > price['today']).astype(int)

# separate training data from testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [4]:
# scale the input data
scaler = StandardScaler()

# Reshape X_train and X_test if they are 1D
if X_train.ndim == 1:
    X_train = X_train.to_numpy().reshape(-1, 1)
if X_test.ndim == 1:
    X_test = X_test.to_numpy().reshape(-1, 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# reshape the input data for CNN-LSTM (samples, timesteps, features)
def create_sequences(data, timesteps):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

timesteps = 5
X_train_reshaped = create_sequences(X_train_scaled, timesteps)
X_test_reshaped = create_sequences(X_test_scaled, timesteps)
y_train = y_train[timesteps - 1:]
y_test = y_test[timesteps - 1:]

In [5]:
from tensorflow.keras import backend as K

def f1_score_2(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [6]:
# define the LSTNet model
input_layer = Input(shape=(timesteps, X_train_reshaped.shape[2]))

# CNN component
cnn = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
cnn = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn)
cnn = TimeDistributed(Dense(64, activation='relu'))(cnn)
cnn = TimeDistributed(Dense(32, activation='relu'))(cnn)
cnn = TimeDistributed(Dense(1))(cnn)
cnn = GlobalAveragePooling1D()(cnn)

# LSTM component
lstm = LSTM(units=256, return_sequences=True)(input_layer)
lstm = LSTM(units=128, return_sequences=True)(lstm)
lstm = TimeDistributed(Dense(64, activation='relu'))(lstm)
lstm = TimeDistributed(Dense(32, activation='relu'))(lstm)
lstm = TimeDistributed(Dense(1))(lstm)
lstm = GlobalAveragePooling1D()(lstm)

# Autoregression (AR) component
ar = Dense(64, activation='linear')(input_layer)
ar = Dense(32, activation='linear')(ar)
ar = Dense(1, activation='linear')(ar)
ar = GlobalAveragePooling1D()(ar)

# Merge all components
merged = Add()([cnn, lstm, ar])
merged = Dense(64, activation='relu')(merged)
merged = Dropout(0.5)(merged)
merged = Dense(32, activation='relu')(merged)
merged = Dropout(0.5)(merged)
merged = Dense(1, activation='sigmoid')(merged)

# Create the model
model = Model(inputs=input_layer, outputs=merged)

# compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=[metrics.BinaryAccuracy(), 
                                                                                        metrics.Precision(), metrics.Recall(), f1_score_2])

In [7]:
model.save("lstnet_c_model.h5")

In [7]:
# train with the training dataset
early_stopping = EarlyStopping(monitor='val_loss', patience=100)
history = model.fit(X_train_reshaped, y_train, epochs=1000, batch_size=50, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/1000
55/55 [==============================] - 16s 68ms/step - loss: 0.6915 - binary_accuracy: 0.5383 - precision: 0.5483 - recall: 0.8814 - f1_score_2: 0.6770 - val_loss: 0.6906 - val_binary_accuracy: 0.5615 - val_precision: 0.5615 - val_recall: 1.0000 - val_f1_score_2: 0.7541
Epoch 2/1000
55/55 [==============================] - 1s 26ms/step - loss: 0.6913 - binary_accuracy: 0.5428 - precision: 0.5449 - recall: 0.9912 - f1_score_2: 0.7052 - val_loss: 0.6894 - val_binary_accuracy: 0.5615 - val_precision: 0.5615 - val_recall: 1.0000 - val_f1_score_2: 0.7541
Epoch 3/1000
55/55 [==============================] - 1s 24ms/step - loss: 0.6920 - binary_accuracy: 0.5450 - precision: 0.5459 - recall: 0.9953 - f1_score_2: 0.7070 - val_loss: 0.6887 - val_binary_accuracy: 0.5615 - val_precision: 0.5615 - val_recall: 1.0000 - val_f1_score_2: 0.7541
Epoch 4/1000
55/55 [==============================] - 1s 25ms/step - loss: 0.6896 - binary_accuracy: 0.5465 - precision: 0.5465 - recall: 1.0000

In [8]:
# predict with testing dataset
y_pred_prob = model.predict(X_test_reshaped)

# convert probabilities to binary class labels
y_pred = (y_pred_prob >= 0.5).astype(int)

24/24 [==============================] - 2s 8ms/step


In [9]:
# evaluate the prediction performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))

Accuracy: 0.517379679144385
Precision: 0.516042780748663
Recall: 0.517426273458445
F1-score: 0.5167336010709505


In [10]:
time = df['timestamp']
time_train, time_test = train_test_split(time, test_size=0.2, shuffle=False)
time_test = time_test[timesteps-1:]

# Flatten y_pred to be a 1-dimensional array
y_pred_flat = y_pred.flatten()

# Create a DataFrame with columns time_test, y_test, and y_pred
pred_res = pd.DataFrame({'date': time_test, 'actual': y_test.values, 'prediction': y_pred_flat, 'value': price['today'][-748:]})

pred_res.to_csv('pred/lstnet_uni_data.csv', index=False)